In [1]:
import pandas as pd
import numpy as np
import datetime as dt

# Cleaning the data

dataset = pd.read_excel("/Users/linus/Documents/Learning Files/Cass/Term 3/SMM799 ARP/mas_car_00_18_2019_adjusted.xlsx")

df = dataset[['Date Announced', 'Original Date Announced', 'Standard Industry Classification Code',
            'Same_industry', 'CAR(-1, +1)', 'CAR(-2, 2)', 'crossborderdeal', 'cashpayment', 'leverage',
            'Acquiror Market Value 4 Weeks Prior to Announcement ($mil).1', 'Deal Attitude', 'Deal Status',
            'Percentage of Shares Acquired']]
df['Date (years)'] = pd.to_datetime(df['Date Announced'], format="%Y%m%d").dt.year

# rename the columns
column_mapping = {'Standard Industry Classification Code': 'SIC',
            'Same_industry': 'Horizontal', 'CAR(-2, 2)': 'CAR(-2, +2)',
            'crossborderdeal': 'Crossborder', 'cashpayment': 'Cash', 'leverage': 'Leverage',
            'Acquiror Market Value 4 Weeks Prior to Announcement ($mil).1': 'Acquiror size'}

df = df.rename(mapper=column_mapping, axis=1)

# drop date announced, original date announced
df = df.drop(['Date Announced', 'Original Date Announced'], axis=1)

# save new dataframe as a .csv file
df.to_csv('ARP_compressed_dataset.csv', index=False)

# Conducting empirical analysis

In [2]:
# reimport the data set
df = pd.read_csv('ARP_compressed_dataset.csv')
df

,SIC,Horizontal,"CAR(-1, +1)","CAR(-2, +2)",Crossborder,Cash,Leverage,Acquiror size,Deal Attitude,Deal Status,Percentage of Shares Acquired,Date (years)
0,5961,0,-0.001608,0.009858,0,0,0.456346,140.433,Friendly,Completed,36,2013
1,5961,0,-0.018310,-0.034374,1,1,0.4521884,555.630,Friendly,Completed,100,2008
2,5961,0,-0.007620,-0.015071,1,1,0.344419,426.282,Friendly,Completed,100,2006
3,5961,0,0.211841,0.146537,1,1,0.3044112,330.046,Friendly,Completed,100,2014
4,6020,0,0.001937,0.048450,1,0,0.9189009,57.730,Friendly,Completed,100,2013
...,...,...,...,...,...,...,...,...,...,...,...,...
20371,6211,1,0.027444,0.020493,1,0,0.7598003,1333.872,Friendly,Pending,.,2008
20372,6211,0,0.100135,0.151409,1,0,0.7244874,667.857,Friendly,Completed,100,2009
20373,6211,0,0.039568,0.030297,1,0,0.7128826,1446.661,Friendly,Completed,100,2007
20374,7372,0,-0.029043,0.003318,1,0,0.295371,355.064,Friendly,Completed,100,2006


In [3]:
# categorise deals in terms of deal attitude
# remove deal attitude that is not applicable and unsolicit
df = df[df.loc[:, 'Deal Attitude'] != 'Not Appl.']
df = df[df.loc[:, 'Deal Attitude'] != 'Unsolic.']

# assign 1 to friendly deals and 0 to other deals
df.loc[:, 'Deal Attitude'].replace(to_replace={'Friendly': 1, 'Neutral': 0, 'Hostile': 0}, inplace=True)
df.reset_index(inplace=True, drop=True)

In [4]:
# convert data in acquired ownership into float
df = df[df['Percentage of Shares Acquired'] != '.']
df['Percentage of Shares Acquired'] = df['Percentage of Shares Acquired'].astype(np.float64)

In [5]:
# only select the completed deals
df = df[df['Deal Status'] == 'Completed']

# only select the deals that over 50% ownership is acquired
df = df[df['Percentage of Shares Acquired'] >= 50]

# drop these two columns
df.drop(columns=['Deal Status', 'Percentage of Shares Acquired'], inplace=True)

In [6]:
# categorise high-tech companies in terms of the SIC code
# http://www.selfcraft.net/ATSELF/TDef.htm
df['SIC'] = df['SIC'].astype(str)

high_tech_sic = [283, 357, 366, 367, 382, 384, 481, 482, 489, 737, 873]
ht_df = pd.DataFrame()

for sic in high_tech_sic:    
    ht_df = pd.concat([ht_df, df[df['SIC'].str.startswith(f'{sic}')]])

ht_df.reset_index(inplace=True, drop=True)
ht_df

,SIC,Horizontal,"CAR(-1, +1)","CAR(-2, +2)",Crossborder,Cash,Leverage,Acquiror size,Deal Attitude,Date (years)
0,2834,0,-0.051517,-0.064767,1,0,0.0557578,38.811,1,2011
1,2834,0,-0.026992,-0.004153,1,1,0.5858068,2100.950,1,2015
2,2834,1,0.381402,0.370299,1,0,0.5700396,496.915,1,2014
3,2836,1,-0.014236,-0.021475,1,0,0.3172007,72.536,1,2008
4,2834,1,0.575205,0.456932,0,0,1.0115591,8.095,1,2006
...,...,...,...,...,...,...,...,...,...,...
5290,8734,0,0.012849,0.018824,1,0,0.3650845,790.984,1,2013
5291,8734,0,0.058681,0.035654,0,0,0.5709707,611.080,1,2008
5292,8734,0,0.077810,0.083322,1,0,0.3597673,830.720,1,2015
5293,8734,0,0.006909,-0.020360,1,0,0.3319166,358.459,1,2010
